In [1]:
import mat73
import pandas as pd
import numpy as np
import scipy.io
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler

In [2]:
import os

DATA_DIR = ''
if 'google.colab' not in str(get_ipython()) and "anuja" in os.environ.get('USER'):
    DATA_DIR = 'data/'
    

## Sensor Level

In [3]:
foof = pd.read_csv(DATA_DIR+"/sensor_intercept_slope.csv")
columns = [['Electrode %d - Intercept'%(i), 'Electrode %d - Slope'%(i)] for i in range(1, 106)]
columns = list(np.array(columns).flatten())
columns.append('IDs')
foof.columns = columns
foof

,Electrode 1 - Intercept,Electrode 1 - Slope,Electrode 2 - Intercept,Electrode 2 - Slope,Electrode 3 - Intercept,Electrode 3 - Slope,Electrode 4 - Intercept,Electrode 4 - Slope,Electrode 5 - Intercept,Electrode 5 - Slope,...,Electrode 101 - Slope,Electrode 102 - Intercept,Electrode 102 - Slope,Electrode 103 - Intercept,Electrode 103 - Slope,Electrode 104 - Intercept,Electrode 104 - Slope,Electrode 105 - Intercept,Electrode 105 - Slope,IDs
0,1.610769,1.793923,1.639958,1.803579,1.622763,1.867163,1.596286,1.868653,1.459489,1.880624,...,1.637747,1.472414,1.775967,1.534612,1.803604,1.575800,1.869977,0.986272,1.825774,NDARAA075AMK
1,1.572269,1.762184,1.615888,1.938907,1.515331,1.878222,1.577389,1.888830,1.645799,1.892993,...,1.698141,1.698537,1.854421,1.486611,1.755794,1.517099,1.845983,1.486650,1.888544,NDARAA112DMH
2,1.414507,1.818785,1.504383,1.902537,1.563395,1.936381,1.554663,2.085351,1.566065,2.106747,...,1.847331,1.417519,1.869143,1.323095,1.838333,1.496008,1.895424,1.593155,2.095749,NDARAA117NEJ
3,0.561062,1.068994,0.333554,0.991180,0.555779,1.304169,0.665864,1.491464,0.524828,1.573271,...,1.464311,0.576556,1.265274,0.199181,0.863242,0.246682,1.010393,0.703331,1.724831,NDARAA947ZG5
4,1.262007,1.901401,1.305927,1.924721,1.293914,1.924840,1.226456,1.947274,0.818595,1.905113,...,1.859064,1.158560,1.863190,1.173287,1.773532,1.252168,1.878925,0.918020,1.749441,NDARAA948VFH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2037,0.648706,1.508826,0.591809,1.640814,0.640782,1.665891,0.455759,1.679019,0.866016,1.786340,...,1.536850,0.528776,1.478037,0.466827,1.767683,0.570274,1.664126,1.351549,1.996940,NDARZN277NR6
2038,1.332278,1.729382,1.387863,1.788208,1.452167,1.841329,1.524229,1.976239,1.480653,2.022584,...,1.876485,1.214977,1.795080,1.456392,1.835807,1.399127,1.795906,1.380795,2.036327,NDARZN578YDP
2039,0.746123,1.324138,0.653335,1.285734,0.564490,1.271931,0.375903,1.245331,0.258508,1.206112,...,1.307891,0.691917,1.310004,0.604437,1.283587,0.564744,1.273098,0.339229,1.050644,NDARZN610GTY
2040,0.976055,1.441830,0.823197,1.407064,0.879368,1.464799,0.902699,1.512205,1.012450,1.505105,...,1.341166,0.598687,1.282644,0.697733,1.421177,0.753861,1.457204,0.781225,1.470061,NDARZN677EYE


In [4]:
data = scipy.io.loadmat(DATA_DIR+'x.mat')  
df = pd.DataFrame(data['x'].reshape((data['x'].shape[0], -1)))

# sparsing
# df = np.array(df).reshape(data['x'].shape)
# df_sparsed = np.concatenate([np.expand_dims(df[:,:,i:i+2].mean(axis = 2), axis = 2) for i in range(0, data['x'].shape[2]-2, 2)], axis = 2)
# df = pd.DataFrame(df_sparsed.reshape((df_sparsed.shape[0], -1)))
# print(df_sparsed.shape)

#scaling
norm = MinMaxScaler().fit(df)
df = norm.transform(df)
df = pd.DataFrame(df.reshape((df.shape[0], -1)))

columns = np.asarray([['Electrode %d - %.1f Hz'%(i+1, j)] for i in range(105) for j in np.arange(1, 40.5, 0.5)])
df.columns = columns
df['IDs'] = foof['IDs']
df

,"(Electrode 1 - 1.0 Hz,)","(Electrode 1 - 1.5 Hz,)","(Electrode 1 - 2.0 Hz,)","(Electrode 1 - 2.5 Hz,)","(Electrode 1 - 3.0 Hz,)","(Electrode 1 - 3.5 Hz,)","(Electrode 1 - 4.0 Hz,)","(Electrode 1 - 4.5 Hz,)","(Electrode 1 - 5.0 Hz,)","(Electrode 1 - 5.5 Hz,)",...,"(Electrode 105 - 36.0 Hz,)","(Electrode 105 - 36.5 Hz,)","(Electrode 105 - 37.0 Hz,)","(Electrode 105 - 37.5 Hz,)","(Electrode 105 - 38.0 Hz,)","(Electrode 105 - 38.5 Hz,)","(Electrode 105 - 39.0 Hz,)","(Electrode 105 - 39.5 Hz,)","(Electrode 105 - 40.0 Hz,)",IDs
0,1.165485e-11,2.269649e-10,3.808305e-09,5.353586e-08,7.565996e-07,0.000011,0.000165,0.001702,0.011524,0.050167,...,4.648661e-09,1.879936e-09,7.239990e-10,2.777809e-10,1.068953e-10,4.127468e-11,1.599129e-11,6.216467e-12,2.424931e-12,NDARAA075AMK
1,5.743954e-02,2.460472e-01,4.457226e-01,3.251448e-01,1.054350e-01,0.044332,0.159810,0.238870,0.150660,0.181196,...,9.069666e-06,2.853855e-06,8.762598e-07,2.795727e-07,9.396118e-08,3.309113e-08,1.205145e-08,4.475759e-09,1.676995e-09,NDARAA112DMH
2,3.181883e-06,6.452939e-05,8.682196e-04,7.378562e-03,4.284223e-02,0.143420,0.305064,0.375769,0.318504,0.217702,...,2.070398e-07,8.101691e-08,2.998807e-08,1.098404e-08,4.008093e-09,1.457652e-09,5.283416e-10,1.908627e-10,6.871824e-11,NDARAA117NEJ
3,2.559403e-01,3.033666e-01,3.496544e-01,3.730970e-01,3.986542e-01,0.360040,0.306996,0.256529,0.263354,0.201815,...,1.293754e-01,1.192672e-01,1.062752e-01,9.575781e-02,8.783538e-02,8.205319e-02,7.806514e-02,7.564052e-02,7.464273e-02,NDARAA947ZG5
4,1.259914e-05,2.329821e-04,2.775527e-03,1.994995e-02,9.327162e-02,0.239264,0.371663,0.320721,0.188774,0.104396,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NDARAA948VFH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2037,1.246152e-05,4.409610e-04,7.140223e-03,5.037035e-02,1.674477e-01,0.221385,0.128560,0.031509,0.010560,0.023736,...,9.096372e-03,6.551053e-03,4.539767e-03,3.166834e-03,2.238763e-03,1.604574e-03,1.165964e-03,8.589819e-04,6.415888e-04,NDARZN277NR6
2038,1.226810e-02,1.509323e-02,1.904249e-02,2.345693e-02,3.051433e-02,0.035372,0.040242,0.040813,0.042936,0.048525,...,9.100567e-04,6.045304e-04,3.849341e-04,2.457844e-04,1.584299e-04,1.031353e-04,6.780565e-05,4.502042e-05,3.018812e-05,NDARZN578YDP
2039,8.275256e-03,1.323160e-02,2.506360e-02,5.001811e-02,1.031895e-01,0.173438,0.252636,0.285581,0.291097,0.280583,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NDARZN610GTY
2040,4.019794e-03,8.629347e-03,1.815964e-02,3.566658e-02,7.071627e-02,0.119430,0.189233,0.255499,0.342001,0.469486,...,5.714034e-04,2.959543e-04,1.450661e-04,7.039729e-05,3.404998e-05,1.642201e-05,7.897488e-06,3.787084e-06,1.810816e-06,NDARZN677EYE


In [5]:
# import matplotlib.pyplot as plt

# plt.plot(np.array(df)[0][79:79+79])
# plt.title('from reshaped data frame')
# plt.show()


# plt.plot(data['x'][0][1])
# plt.title('from x.mat')
# plt.show()


# import pandas
# path = os.path.join(DATA_DIR, 'split_data_csv')
# sensor = pandas.read_csv(os.path.join(path, 'train_features_sensor.csv'))
# plt.plot(np.array(sensor[sensor.IDs.isin(['NDARAA075AMK'])])[0][79:79+79])
# plt.title('from sensor features csv')
# plt.show()

# df.to_csv('X_sensor_noscaling.csv', sep='\t')

In [6]:
beh = pd.read_csv(DATA_DIR+"behaviorals.csv")
print('Before:'+str(beh.shape))

most_common_disorders = ['Attention-Deficit/Hyperactivity Disorder', 'Anxiety Disorders', 'Specific Learning Disorder',
                         'Autism Spectrum Disorder', 'Disruptive', 'Communication Disorder',
                         'Depressive Disorders',  'No Diagnosis Given', 'Other Disorders']

category_columns = ['DX_' + str(i).zfill(2) + '_Cat' for i in range(1, 11)] +\
                   ['DX_' + str(i).zfill(2) + '_Sub' for i in range(1, 11)]

# removing patients with incomplete eval
initial_size = beh.shape[0]
beh = beh[beh.DX_01 != 'No Diagnosis Given: Incomplete Eval']
beh = beh.reset_index(drop=True)
new_size = beh.shape[0]

print('After:'+str(beh.shape))
print('Removing', initial_size - new_size,
      'patients as their evaluations was incomplete.')

Before:(3076, 177)
After:(2939, 177)
Removing 137 patients as their evaluations was incomplete.


In [7]:
no_diagnosis_given = 'No Diagnosis Given'
diagnoses_to_ids = {disorder: i for i, disorder in enumerate(most_common_disorders)}
diagnoses_to_ids

{'Attention-Deficit/Hyperactivity Disorder': 0,
 'Anxiety Disorders': 1,
 'Specific Learning Disorder': 2,
 'Autism Spectrum Disorder': 3,
 'Disruptive': 4,
 'Communication Disorder': 5,
 'Depressive Disorders': 6,
 'No Diagnosis Given': 7,
 'Other Disorders': 8}

In [8]:
def get_disorder(data, row, index):
    disorder = data.iloc[row][category_columns[index]]

    if disorder == 'Neurodevelopmental Disorders':
        disorder = data.iloc[row][category_columns[index + 10]]

    return disorder

order_of_disorders = []
for k in range(beh.shape[0]):
    i = 0
    disorder = get_disorder(beh, k, i)
    disorders_patient = []
    while not pd.isnull(disorder):
        if disorder in diagnoses_to_ids:
            if diagnoses_to_ids[disorder] not in disorders_patient:
                disorders_patient.append(diagnoses_to_ids[disorder])
        else:
            if diagnoses_to_ids['Other Disorders'] not in disorders_patient:
                disorders_patient.append(diagnoses_to_ids['Other Disorders'])
        i += 1
        if i == len(category_columns):
            break
        disorder = get_disorder(beh, k, i)

        
    order_of_disorders.append(disorders_patient)

In [9]:
other_disorders = []
no_diagnosis_given = []
for i in order_of_disorders:
    if 7 in i:
        no_diagnosis_given.append(1)
        i.remove(7)
    else:
        no_diagnosis_given.append(0)
    if 8 in i:
        other_disorders.append(1)
        i.remove(8)
    else:
        other_disorders.append(0)

In [10]:
max_len_order = np.max([len(x) for x in order_of_disorders])

# pad with a new token denoting the pad token
pad_token = len(most_common_disorders)
bod_token = len(most_common_disorders) + 1
eod_token = len(most_common_disorders) + 2

order_of_disorders = [[bod_token] + x + [eod_token] + [pad_token] * (max_len_order - len(x)) for x in order_of_disorders]

order_of_disorders = np.array(order_of_disorders)

classes = np.zeros((len(most_common_disorders),
                    beh.shape[0]), dtype=np.int32)

df_disorders = beh[category_columns]

for i, disorder in enumerate(most_common_disorders):
    mask = df_disorders.select_dtypes(include=[object]). \
        applymap(lambda x: disorder in x if pd.notnull(x) else False)

    disorder_df = df_disorders[mask.any(axis=1)]

    np.add.at(classes[i], disorder_df.index.values, 1)

behaviour_data_columns = beh.columns.values.astype(np.str)

columns_to_drop = behaviour_data_columns[
    np.flatnonzero(np.core.defchararray.find(behaviour_data_columns, 'DX') != -1)]

behaviour_data = beh.drop(columns=columns_to_drop)

for disorder, classification in zip(most_common_disorders, classes):
    behaviour_data[disorder] = classification

behaviour_data['order_diagnoses'] = list(order_of_disorders)

<ipython-input-10-48b714085ca7>:25: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  behaviour_data_columns = beh.columns.values.astype(np.str)


In [11]:
common_disorders = ['Attention-Deficit/Hyperactivity Disorder', 'Anxiety Disorders', 'Specific Learning Disorder',
                         'Autism Spectrum Disorder', 'Disruptive', 'Communication Disorder',
                         'Depressive Disorders']

labels=behaviour_data[["IDs"]+list(common_disorders)]
labels["Other Disorders"] = other_disorders
labels

<ipython-input-11-26c912096f42>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels["Other Disorders"] = other_disorders


,IDs,Attention-Deficit/Hyperactivity Disorder,Anxiety Disorders,Specific Learning Disorder,Autism Spectrum Disorder,Disruptive,Communication Disorder,Depressive Disorders,Other Disorders
0,NDARAA075AMK,0,0,0,0,0,0,0,0
1,NDARAA112DMH,1,0,0,0,1,0,0,1
2,NDARAA117NEJ,1,0,0,0,1,0,0,1
3,NDARAA306NT2,1,1,1,0,0,1,0,1
4,NDARAA504CRN,1,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
2934,NDARZZ007YMP,0,0,0,1,0,0,0,1
2935,NDARZZ740MLM,1,0,0,0,0,0,0,0
2936,NDARZZ810LVF,0,0,0,1,0,1,0,1
2937,NDARZZ830JM7,0,0,0,1,0,0,0,1


In [12]:
add_features = behaviour_data[['Sex', 'Age','IDs']]
add_features.shape

(2939, 3)

In [13]:
df = pd.merge(df, foof, on='IDs', how='inner')
df = pd.merge(df, labels, on='IDs', how='inner')
df = pd.merge(df, add_features, on='IDs', how='inner')
df

,"(Electrode 1 - 1.0 Hz,)","(Electrode 1 - 1.5 Hz,)","(Electrode 1 - 2.0 Hz,)","(Electrode 1 - 2.5 Hz,)","(Electrode 1 - 3.0 Hz,)","(Electrode 1 - 3.5 Hz,)","(Electrode 1 - 4.0 Hz,)","(Electrode 1 - 4.5 Hz,)","(Electrode 1 - 5.0 Hz,)","(Electrode 1 - 5.5 Hz,)",...,Attention-Deficit/Hyperactivity Disorder,Anxiety Disorders,Specific Learning Disorder,Autism Spectrum Disorder,Disruptive,Communication Disorder,Depressive Disorders,Other Disorders,Sex,Age
0,1.165485e-11,2.269649e-10,3.808305e-09,5.353586e-08,7.565996e-07,0.000011,0.000165,0.001702,0.011524,0.050167,...,0,0,0,0,0,0,0,0,1,6.728040
1,5.743954e-02,2.460472e-01,4.457226e-01,3.251448e-01,1.054350e-01,0.044332,0.159810,0.238870,0.150660,0.181196,...,1,0,0,0,1,0,0,1,0,5.545744
2,3.181883e-06,6.452939e-05,8.682196e-04,7.378562e-03,4.284223e-02,0.143420,0.305064,0.375769,0.318504,0.217702,...,1,0,0,0,1,0,0,1,0,7.475929
3,2.559403e-01,3.033666e-01,3.496544e-01,3.730970e-01,3.986542e-01,0.360040,0.306996,0.256529,0.263354,0.201815,...,1,0,1,1,0,0,0,1,0,13.627880
4,1.259914e-05,2.329821e-04,2.775527e-03,1.994995e-02,9.327162e-02,0.239264,0.371663,0.320721,0.188774,0.104396,...,1,0,0,0,0,0,0,1,1,7.982660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1916,5.137044e-06,2.553540e-05,1.165434e-04,4.649161e-04,1.753336e-03,0.005275,0.013941,0.029402,0.057570,0.108259,...,0,1,0,0,0,0,0,0,1,11.629363
1917,1.246152e-05,4.409610e-04,7.140223e-03,5.037035e-02,1.674477e-01,0.221385,0.128560,0.031509,0.010560,0.023736,...,1,1,0,0,0,0,1,0,1,14.878736
1918,8.275256e-03,1.323160e-02,2.506360e-02,5.001811e-02,1.031895e-01,0.173438,0.252636,0.285581,0.291097,0.280583,...,0,0,0,0,0,0,0,1,1,16.379534
1919,4.019794e-03,8.629347e-03,1.815964e-02,3.566658e-02,7.071627e-02,0.119430,0.189233,0.255499,0.342001,0.469486,...,1,0,0,0,0,0,0,0,1,15.029545


In [14]:
behaviour_data_reg = pd.read_csv(DATA_DIR+'AllData.csv')
behaviour_data_reg_ = behaviour_data_reg.rename(columns = {'EID': 'IDs'}, inplace = True)
behaviour_data_reg.columns

Index(['IDs', 'SRS_SCI_T', 'SRS_RRB_T', 'SWAN_IN_Avg', 'SWAN_HY_Avg',
       'SWAN_Avg', 'SCARED_SR_GD', 'SCARED_P_GD', 'WISC_WMI_Sum',
       'WISC_VCI_Sum'],
      dtype='object')

In [15]:
behaviour_data_reg = behaviour_data_reg[['IDs', 'SRS_SCI_T', 'SRS_RRB_T', 'SWAN_IN_Avg', 'SWAN_HY_Avg',
       'SWAN_Avg', 'SCARED_P_GD', 'WISC_WMI_Sum',
       'WISC_VCI_Sum']]
behaviour_data_reg.shape

(2579, 9)

In [16]:
df = pd.merge(df, behaviour_data_reg, on='IDs', how='left')
df.shape


(1921, 8524)

In [17]:
disorders_list = ['Attention-Deficit/Hyperactivity Disorder', 'Anxiety Disorders', 'Specific Learning Disorder',
                         'Autism Spectrum Disorder', 'Disruptive', 'Communication Disorder',
                         'Depressive Disorders', 'Other Disorders']

labels_list = [ 'SRS_SCI_T', 'SRS_RRB_T', 'SWAN_IN_Avg', 'SWAN_HY_Avg', 'SCARED_P_GD', 'WISC_WMI_Sum', 'WISC_VCI_Sum']

x_sensor = pd.DataFrame.copy(df)
x_sensor = x_sensor.drop(disorders_list+labels_list+['SWAN_Avg'], axis=1)

y = df[disorders_list+labels_list]
y["IDs"] = x_sensor["IDs"] #adding IDs for split

print(x_sensor.shape, y.shape)


(1921, 8508) (1921, 16)


<ipython-input-17-d8d537e25934>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y["IDs"] = x_sensor["IDs"] #adding IDs for split


In [18]:
x_sensor.columns

Index([  ('Electrode 1 - 1.0 Hz',),   ('Electrode 1 - 1.5 Hz',),
         ('Electrode 1 - 2.0 Hz',),   ('Electrode 1 - 2.5 Hz',),
         ('Electrode 1 - 3.0 Hz',),   ('Electrode 1 - 3.5 Hz',),
         ('Electrode 1 - 4.0 Hz',),   ('Electrode 1 - 4.5 Hz',),
         ('Electrode 1 - 5.0 Hz',),   ('Electrode 1 - 5.5 Hz',),
       ...
       'Electrode 102 - Intercept',     'Electrode 102 - Slope',
       'Electrode 103 - Intercept',     'Electrode 103 - Slope',
       'Electrode 104 - Intercept',     'Electrode 104 - Slope',
       'Electrode 105 - Intercept',     'Electrode 105 - Slope',
                             'Sex',                       'Age'],
      dtype='object', length=8508)

## Source Level

In [19]:
# ## data loading
# data_source = mat73.loadmat(DATA_DIR+'x_source_new.mat')  
foof_source = pd.read_csv(DATA_DIR+"source_intercept_slope.csv")
foof_source = foof_source.drop('Unnamed: 0', 1)
columns = [['Parcel %d - Intercept'%(i), 'Parcel %d - Slope'%(i)] for i in range(1, 69)]
columns = list(np.array(columns).flatten())
columns.append('IDs')
foof_source.columns = columns

In [20]:
pd.read_csv('params_source.csv')

,p_1_intercept,p_1_slope,p_2_intercept,p_2_slope,p_3_intercept,p_3_slope,p_4_intercept,p_4_slope,p_5_intercept,p_5_slope,...,p_64_intercept,p_64_slope,p_65_intercept,p_65_slope,p_66_intercept,p_66_slope,p_67_intercept,p_67_slope,p_68_intercept,p_68_slope
0,0.160123,1.362617,-0.002037,1.427172,0.810964,1.391837,0.745124,1.378194,0.527491,1.433532,...,-0.097145,1.519565,1.004265,1.450747,0.956578,1.456952,0.611944,1.503932,0.282555,1.441946
1,0.988915,1.330636,1.151667,1.546990,1.479442,1.608810,1.342563,1.559859,0.460923,1.434832,...,1.149346,1.461904,1.449481,1.480738,0.889122,1.282076,0.950479,1.138697,1.372408,1.505823
2,1.455004,1.515717,0.988018,1.367703,2.029333,1.763131,2.086803,1.763987,1.320758,1.585413,...,1.130052,1.502295,1.596192,1.448922,1.180667,1.199036,1.658822,1.619147,1.182087,1.395998
3,0.796931,1.547997,0.016022,0.956344,0.638462,1.222323,0.698921,1.395078,-0.130122,1.156477,...,-0.241526,0.998001,0.079507,1.285272,0.471817,1.175649,0.617125,1.418717,0.401675,1.222953
4,-0.627481,1.123518,-0.255195,1.273727,0.197328,1.344738,0.282533,1.387126,-0.808075,1.077384,...,0.001076,1.363285,0.049249,1.207528,-0.118998,1.296786,-0.253839,1.176320,-0.033039,1.318181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2037,-0.561398,1.060676,-0.680878,1.241504,0.177737,1.110332,0.141657,1.116719,-0.347975,1.110417,...,-0.934720,1.153422,0.182297,1.016177,0.177069,1.042335,-0.124226,1.165734,-0.418218,1.085482
2038,1.178082,1.344137,1.127791,1.414553,1.802019,1.642233,1.680568,1.593067,0.891970,1.502417,...,1.305561,1.576864,2.119400,1.755767,1.780714,1.576273,1.551271,1.384801,1.587596,1.466845
2039,-0.394953,1.348060,-0.918028,1.088161,-0.134448,1.122479,-0.268877,1.111450,-0.288241,1.206044,...,-0.891403,1.099186,-0.022279,1.275160,-0.316072,1.201135,-0.249886,1.300831,-0.656477,1.232983
2040,-0.347506,0.870524,0.144745,1.257984,0.636052,0.952351,0.745472,0.988167,-0.353721,0.937682,...,0.254825,1.278362,0.544927,0.883756,0.788643,1.038478,-0.372208,0.747925,0.414824,1.251458


In [21]:
foof_source

,Parcel 1 - Intercept,Parcel 1 - Slope,Parcel 2 - Intercept,Parcel 2 - Slope,Parcel 3 - Intercept,Parcel 3 - Slope,Parcel 4 - Intercept,Parcel 4 - Slope,Parcel 5 - Intercept,Parcel 5 - Slope,...,Parcel 64 - Slope,Parcel 65 - Intercept,Parcel 65 - Slope,Parcel 66 - Intercept,Parcel 66 - Slope,Parcel 67 - Intercept,Parcel 67 - Slope,Parcel 68 - Intercept,Parcel 68 - Slope,IDs
0,0.160123,1.362617,-0.002037,1.427172,0.810964,1.391837,0.745124,1.378194,0.527491,1.433532,...,1.519565,1.004265,1.450747,0.956578,1.456952,0.611944,1.503932,0.282555,1.441946,NDARAA075AMK
1,0.988915,1.330636,1.151667,1.546990,1.479442,1.608810,1.342563,1.559859,0.460923,1.434832,...,1.461904,1.449481,1.480738,0.889122,1.282076,0.950479,1.138697,1.372408,1.505823,NDARAA112DMH
2,1.455004,1.515717,0.988018,1.367703,2.029333,1.763131,2.086803,1.763987,1.320758,1.585413,...,1.502295,1.596192,1.448922,1.180667,1.199036,1.658822,1.619147,1.182087,1.395998,NDARAA117NEJ
3,0.796931,1.547997,0.016022,0.956344,0.638462,1.222323,0.698921,1.395078,-0.130122,1.156477,...,0.998001,0.079507,1.285272,0.471817,1.175649,0.617125,1.418717,0.401675,1.222953,NDARAA947ZG5
4,-0.627481,1.123518,-0.255195,1.273727,0.197328,1.344738,0.282533,1.387126,-0.808075,1.077384,...,1.363285,0.049249,1.207528,-0.118998,1.296786,-0.253839,1.176320,-0.033039,1.318181,NDARAA948VFH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2037,-0.561398,1.060676,-0.680878,1.241504,0.177737,1.110332,0.141657,1.116719,-0.347975,1.110417,...,1.153422,0.182297,1.016177,0.177069,1.042335,-0.124226,1.165734,-0.418218,1.085482,NDARZN277NR6
2038,1.178082,1.344137,1.127791,1.414553,1.802019,1.642233,1.680568,1.593067,0.891970,1.502417,...,1.576864,2.119400,1.755767,1.780714,1.576273,1.551271,1.384801,1.587596,1.466845,NDARZN578YDP
2039,-0.394953,1.348060,-0.918028,1.088161,-0.134448,1.122479,-0.268877,1.111450,-0.288241,1.206044,...,1.099186,-0.022279,1.275160,-0.316072,1.201135,-0.249886,1.300831,-0.656477,1.232983,NDARZN610GTY
2040,-0.347506,0.870524,0.144745,1.257984,0.636052,0.952351,0.745472,0.988167,-0.353721,0.937682,...,1.278362,0.544927,0.883756,0.788643,1.038478,-0.372208,0.747925,0.414824,1.251458,NDARZN677EYE


In [22]:
data = mat73.loadmat(DATA_DIR+'x_source_new.mat')  
df2 = pd.DataFrame(data['x'].reshape((data['x'].shape[0], -1)))

# sparsing
# df2 = np.array(df2).reshape(data['x'].shape) 
# df2_sparsed = np.concatenate([np.expand_dims(df2[:,:,i:i+2].mean(axis = 2), axis = 2) for i in range(0, data['x'].shape[2]-2, 2)], axis = 2)
# df2 = pd.DataFrame(df2_sparsed.reshape((df2_sparsed.shape[0], -1)))

#scaling
norm = MinMaxScaler().fit(df2)
df2 = norm.transform(df2)
df2 = pd.DataFrame(df2.reshape((df2.shape[0], -1)))

columns = np.asarray([['Parcel %d - %.1f Hz'%(i+1, j)] for i in range(68) for j in np.arange(1, 40.5, 0.5)])
df2.columns = columns
df2['IDs'] = foof['IDs']

df_source = pd.DataFrame.copy(df2)
df_source

,"(Parcel 1 - 1.0 Hz,)","(Parcel 1 - 1.5 Hz,)","(Parcel 1 - 2.0 Hz,)","(Parcel 1 - 2.5 Hz,)","(Parcel 1 - 3.0 Hz,)","(Parcel 1 - 3.5 Hz,)","(Parcel 1 - 4.0 Hz,)","(Parcel 1 - 4.5 Hz,)","(Parcel 1 - 5.0 Hz,)","(Parcel 1 - 5.5 Hz,)",...,"(Parcel 68 - 36.0 Hz,)","(Parcel 68 - 36.5 Hz,)","(Parcel 68 - 37.0 Hz,)","(Parcel 68 - 37.5 Hz,)","(Parcel 68 - 38.0 Hz,)","(Parcel 68 - 38.5 Hz,)","(Parcel 68 - 39.0 Hz,)","(Parcel 68 - 39.5 Hz,)","(Parcel 68 - 40.0 Hz,)",IDs
0,3.946024e-12,1.014563e-10,2.278631e-09,3.896326e-08,4.384222e-07,4.202851e-06,2.965059e-05,2.216488e-04,1.395206e-03,6.033762e-03,...,5.936173e-13,1.770432e-14,3.353221e-16,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NDARAA075AMK
1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.224459e-01,2.647155e-01,3.415365e-01,2.055876e-01,7.956660e-02,1.435611e-02,1.344153e-03,6.520693e-05,1.536517e-06,NDARAA112DMH
2,2.592388e-04,5.485259e-02,2.285214e-01,1.633807e-02,1.732562e-05,3.528198e-10,1.400551e-13,5.924346e-12,2.111799e-10,6.045110e-09,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NDARAA117NEJ
3,6.473798e-03,4.237279e-02,2.469837e-01,6.074552e-01,5.067741e-01,2.002160e-01,1.406901e-01,5.141211e-01,6.969751e-01,2.480546e-01,...,6.864238e-02,3.612362e-02,1.014439e-01,2.427796e-01,4.359840e-01,4.177373e-01,2.375282e-01,8.002380e-02,1.497544e-02,NDARAA947ZG5
4,1.973195e-01,2.846185e-01,3.990516e-01,4.740013e-01,4.122722e-01,3.399388e-01,2.295374e-01,1.827457e-01,1.363263e-01,7.775023e-02,...,7.337468e-02,7.202776e-02,8.088370e-02,1.416638e-01,8.213267e-02,6.682195e-02,6.553430e-02,6.421162e-02,5.889123e-02,NDARAA948VFH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2037,2.271181e-04,7.503756e-03,8.107694e-02,2.497005e-01,1.894589e-01,4.585309e-02,3.078721e-03,2.450861e-04,1.042577e-03,4.487430e-03,...,4.596463e-08,1.633892e-08,5.645057e-09,1.702897e-09,6.185549e-10,1.961710e-10,6.046338e-11,1.808344e-11,4.919574e-12,NDARZN277NR6
2038,1.762258e-07,4.231697e-06,7.345389e-05,8.001877e-04,4.725649e-03,1.958605e-02,4.922580e-02,1.082838e-01,1.685304e-01,1.665922e-01,...,8.984474e-12,2.686631e-12,7.853244e-13,2.016668e-13,6.267601e-14,1.720464e-14,4.522994e-15,1.247478e-15,2.071923e-16,NDARZN578YDP
2039,0.000000e+00,1.152073e-15,6.198946e-14,2.564923e-12,6.779087e-11,1.481919e-09,2.314310e-08,3.717622e-07,4.881500e-06,4.274859e-05,...,1.933085e-01,1.993560e-01,2.034061e-01,1.844683e-01,2.050707e-01,2.026321e-01,1.980931e-01,1.912997e-01,1.710837e-01,NDARZN610GTY
2040,1.886881e-08,2.255111e-07,2.438372e-06,2.078291e-05,1.206760e-04,6.180048e-04,2.411238e-03,1.031976e-02,3.850149e-02,1.021646e-01,...,8.062438e-07,3.198475e-07,1.235721e-07,4.177004e-08,1.703640e-08,6.079357e-09,2.112703e-09,7.139158e-10,2.199123e-10,NDARZN677EYE


In [23]:
x_source = pd.merge(df_source, foof_source, on='IDs', how='inner')
x_source = pd.merge(x_source, add_features, on='IDs', how='inner')
x_source.shape

(1921, 5511)

In [24]:
# a = np.array(xsourcesplit[xsourcesplit.columns[:79]])
# avg = np.average(a, axis=0)
# plt.plot(avg)
# plt.title('from xsource')
# plt.show()


# a = np.array(source[source.columns[:79]])
# avg = np.average(a, axis=0)
# plt.plot(avg)
# plt.title('from xsource csv')
# plt.show()

## Split

In [25]:
train_features_sensor, test_features_sensor, train_labels, test_labels = train_test_split(x_sensor, y, test_size=0.25, shuffle=True)

In [26]:
train_features_source = x_source[x_source.IDs.isin(train_features_sensor.IDs)]
test_features_source = x_source[x_source.IDs.isin(test_features_sensor.IDs)]

In [27]:
# classfication
train_labels_classification = train_labels[disorders_list+["IDs"]]
test_labels_classification = test_labels[disorders_list+["IDs"]]

# regression
train_labels_regression = train_labels[labels_list+["IDs"]]
test_labels_regression = test_labels[labels_list+["IDs"]]
test_labels_regression = test_labels_regression.dropna()

In [28]:
# modify test feature set based on column drop
test_features_sensor_regression = test_features_sensor[test_features_sensor.IDs.isin(test_labels_regression.IDs)]
test_features_source_regression = test_features_source[test_features_source.IDs.isin(test_labels_regression.IDs)]

In [29]:
test_features_sensor_classification = pd.DataFrame.copy(test_features_sensor)
test_features_source_classification = pd.DataFrame.copy(test_features_source)

In [30]:
# sanity checks
assert sorted(test_features_source_classification.IDs) == sorted(test_features_sensor_classification.IDs)
assert sorted(test_features_source_regression.IDs) == sorted(test_features_sensor_regression.IDs)
assert sorted(train_features_source.IDs) == sorted(train_features_sensor.IDs)

In [31]:
# saving to csv (hacky fix)

path = os.path.join(DATA_DIR, 'split_data')

#saving
if not(os.path.exists(path)):
    os.mkdir(path)
train_features_source.to_pickle(os.path.join(path, 'train_features_source.pkl'))
test_features_source_classification.to_pickle(os.path.join(path, 'test_features_source_classification.pkl'))
train_features_sensor.to_pickle(os.path.join(path, 'train_features_sensor.pkl'))
test_features_sensor_classification.to_pickle(os.path.join(path, 'test_features_sensor_classification.pkl'))
train_labels_classification.to_pickle(os.path.join(path, 'train_labels_classification.pkl'))
test_labels_classification.to_pickle(os.path.join(path, 'test_labels_classification.pkl'))
test_features_source_regression.to_pickle(os.path.join(path, 'test_features_source_regression.pkl'))
test_features_sensor_regression.to_pickle(os.path.join(path, 'test_features_sensor_regression.pkl'))
train_labels_regression.to_pickle(os.path.join(path, 'train_labels_regression.pkl'))
test_labels_regression.to_pickle(os.path.join(path, 'test_labels_regression.pkl'))

# save to csv
import pickle as pkl
import pandas as pd

new_path = os.path.join(DATA_DIR, 'split_data_csv')
for file in os.listdir(path):
    df = pd.read_pickle(os.path.join(path, file))
    df.to_csv(os.path.join(new_path, file.split('.')[0]+'.csv'), index=False)

In [32]:
#remove IDs
train_features_source = train_features_source[train_features_source.columns.difference(['IDs'])]
test_features_source_classification = test_features_source_classification[test_features_source_classification.columns.difference(['IDs'])]
train_features_sensor = train_features_sensor[train_features_sensor.columns.difference(['IDs'])]
test_features_sensor_classification = test_features_sensor_classification[test_features_sensor_classification.columns.difference(['IDs'])]
test_features_source_regression = test_features_source_regression[test_features_source_regression.columns.difference(['IDs'])]
test_features_sensor_regression = test_features_sensor_regression[test_features_sensor_regression.columns.difference(['IDs'])]
test_labels_regression = test_labels_regression[test_labels_regression.columns.difference(['IDs'])]
train_labels_classification = train_labels_classification[train_labels_classification.columns.difference(['IDs'])]
test_labels_classification = test_labels_classification[test_labels_classification.columns.difference(['IDs'])]
train_labels_regression = train_labels_regression[train_labels_regression.columns.difference(['IDs'])]

In [33]:
# ordering labels
train_labels_regression = train_labels_regression.reindex(sorted(train_labels_regression.columns), axis=1)
test_labels_regression = test_labels_regression.reindex(sorted(test_labels_regression.columns), axis=1)

In [34]:
print("CLASSIFICATION\n")
print("Features:")
print('Source:', train_features_source.shape, test_features_source_classification.shape)
print('Sensor:', train_features_sensor.shape, test_features_sensor_classification.shape)
print("Labels:")
print(train_labels_classification.shape, test_labels_classification.shape)

print('\n')

print("REGRESSION\n")
print("Features:")
print('Source:', train_features_source.shape, test_features_source_regression.shape)
print('Sensor:', train_features_sensor.shape, test_features_sensor_regression.shape)
print("Labels:")
print(train_labels_regression.shape, test_labels_regression.shape)


CLASSIFICATION

Features:
Source: (1440, 5510) (481, 5510)
Sensor: (1440, 8507) (481, 8507)
Labels:
(1440, 8) (481, 8)


REGRESSION

Features:
Source: (1440, 5510) (249, 5510)
Sensor: (1440, 8507) (249, 8507)
Labels:
(1440, 7) (249, 7)


In [35]:
path = os.path.join(DATA_DIR, 'split_data')

#overwriting previous save without IDs
if not(os.path.exists(path)):
    os.mkdir(path)
train_features_source.to_pickle(os.path.join(path, 'train_features_source.pkl'))
test_features_source_classification.to_pickle(os.path.join(path, 'test_features_source_classification.pkl'))
train_features_sensor.to_pickle(os.path.join(path, 'train_features_sensor.pkl'))
test_features_sensor_classification.to_pickle(os.path.join(path, 'test_features_sensor_classification.pkl'))
train_labels_classification.to_pickle(os.path.join(path, 'train_labels_classification.pkl'))
test_labels_classification.to_pickle(os.path.join(path, 'test_labels_classification.pkl'))
test_features_source_regression.to_pickle(os.path.join(path, 'test_features_source_regression.pkl'))
test_features_sensor_regression.to_pickle(os.path.join(path, 'test_features_sensor_regression.pkl'))
train_labels_regression.to_pickle(os.path.join(path, 'train_labels_regression.pkl'))
test_labels_regression.to_pickle(os.path.join(path, 'test_labels_regression.pkl'))